# Face detection and recognition inference pipeline

The following example illustrates how to use the `facenet_pytorch` python package to perform face detection and recogition on an image dataset using an Inception Resnet V1 pretrained on the VGGFace2 dataset.

The following Pytorch methods are included:
* Datasets
* Dataloaders
* GPU/CPU processing

In [11]:
# from . import mtcnn
from mtcnn import MTCNN
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import pandas as pd
import os

workers = 0 if os.name == 'nt' else 4

#### Determine if an nvidia GPU is available

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


#### Define MTCNN module

Default params shown for illustration, but not needed. Note that, since MTCNN is a collection of neural nets and other code, the device must be passed in the following way to enable copying of objects when needed internally.

See `help(MTCNN)` for more details.

In [14]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [15]:
def collate_fn(x):
    return x[0]

dataset = datasets.ImageFolder('../data/test_images')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=collate_fn, num_workers=workers)

#### Perfom MTCNN facial detection

Iterate through the DataLoader object and detect faces and associated detection probabilities for each. The `MTCNN` forward method returns images cropped to the detected face, if a face was detected. By default only a single detected face is returned - to have `MTCNN` return all detected faces, set `keep_all=True` when creating the MTCNN object above.

To obtain bounding boxes rather than cropped face images, you can instead call the lower-level `mtcnn.detect()` function. See `help(mtcnn.detect)` for details.

In [73]:
from torchvision.transforms import ToTensor

aligned = []
names = []
for x, y in loader:
    # Convert to Tensor and normalize properly
    # x_tensor = torch.tensor(x, dtype=torch.float32).permute(1,0,2).unsqueeze(0).to(device)
    x_tensor = ToTensor()(x).unsqueeze(0).to(device)
    print(f"Tensor shape: {x_tensor.shape}, range: [{x_tensor.min()}, {x_tensor.max()}]")

    # Enable gradient computation
    # x_tensor.requires_grad = True
    
    x_tensor = x_tensor.permute(0, 2, 3, 1)*255.0
    
    x_tensor.requires_grad = True

    x_aligned, prob = mtcnn(x_tensor, return_prob=True)
    
    if x_aligned is not None:
        if isinstance(prob, np.ndarray):
            prob = prob.item()  # Extract scalar if it's a single-element array
        print('Face detected with probability: {:8f}'.format(prob))
        
            
        print(x_tensor)
        
        # Convert prob to a tensor (if not already) and retain it in the computation graph
        prob_tensor = torch.tensor(prob, device=device, requires_grad=True)
        
        # Define the loss as the negative probability
        loss = -prob_tensor

        # Backpropagate to compute gradients
        loss.backward()
        
        print(prob_tensor.grad)
        
        # FGSM attack: Add perturbation to the input image
        epsilon = 0.01  # Small perturbation value
        perturbed_image = x_tensor + epsilon * x_tensor.grad.sign()

        # Ensure the perturbed image stays in valid range
        # perturbed_image = torch.clamp(perturbed_image, 0, 1)
        
        aligned.append(x_aligned)
        names.append(dataset.idx_to_class[y])

Tensor shape: torch.Size([1, 3, 2048, 1665]), range: [0.0, 1.0]
Face detected with probability: 0.999983
tensor([[[[233., 216., 208.],
          [233., 216., 208.],
          [232., 215., 207.],
          ...,
          [228., 211., 203.],
          [227., 213., 204.],
          [228., 211., 203.]],

         [[233., 216., 208.],
          [232., 215., 207.],
          [232., 215., 207.],
          ...,
          [228., 211., 203.],
          [228., 214., 205.],
          [229., 212., 204.]],

         [[232., 215., 207.],
          [232., 215., 207.],
          [231., 214., 206.],
          ...,
          [229., 212., 204.],
          [227., 213., 204.],
          [230., 213., 205.]],

         ...,

         [[ 21.,  20.,  18.],
          [ 22.,  21.,  19.],
          [ 23.,  22.,  20.],
          ...,
          [237., 220., 210.],
          [236., 219., 209.],
          [238., 221., 211.]],

         [[ 20.,  19.,  17.],
          [ 20.,  19.,  17.],
          [ 22.,  21.,  19.],
  

AttributeError: 'NoneType' object has no attribute 'sign'